In [3]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
import torch
from keras.preprocessing.image import ImageDataGenerator
from torch.utils.data import Dataset
from torchvision import transforms,datasets 
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import cv2
from sklearn.preprocessing import MultiLabelBinarizer
import random
from tensorflow.keras.applications import ResNet50
import csv
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score

In [37]:
class MultiLabelImageDataset(keras.utils.Sequence):
    def __init__(self, image_dir, label_file, num_classes, batch_size=32, shuffle=True, subset='train'):
        self.image_dir = image_dir
        self.num_classes = num_classes
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.subset = subset
        self.data = []
        
        # Load labels from the CSV file
        with open(label_file, 'r') as f:
            reader = csv.reader(f)
            for row in reader:
                image_id, labels_str = row
                labels = [int(label) for label in labels_str.split()]
                self.data.append((os.path.join(image_dir, f"{image_id}.jpg"), labels))
        
        # Split the data into train and test sets
        random.shuffle(self.data)
        split_idx = int(len(self.data) * 0.8)  # 80% for training, 20% for testing
        if self.subset == 'train':
            self.data = self.data[:split_idx]
        else:
            self.data = self.data[split_idx:]
        
        self.indexes = np.arange(len(self.data))
        if self.shuffle:
            np.random.shuffle(self.indexes)
    
    def __len__(self):
        return int(np.ceil(len(self.data) / self.batch_size))
    
    def __getitem__(self, idx):
        batch_indexes = self.indexes[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_images = []
        batch_targets = []
        for i in batch_indexes:
            image_path, labels = self.data[i]
            
            # Load the image
            img = Image.open(image_path)
            
            # Resize the image to 224x224
            img = img.resize((224, 224))
            
            # Convert image to numpy array
            img = np.array(img)
            
            # Normalize the image (important for ResNet)
            img = keras.applications.resnet50.preprocess_input(img)
            
            # Convert labels to a one-hot encoded numpy array
            target = np.zeros(self.num_classes)
            for label in labels:
                target[label - 1] = 1
            
            batch_images.append(img)
            batch_targets.append(target)
        
        batch_images = np.array(batch_images)
        batch_targets = np.array(batch_targets)
        
        return batch_images, batch_targets
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)
            
    def get_labels(self):
        return [labels for _, labels in self.data]

In [38]:
label_file = 'train1.csv'
image_dir = 'data/'
num_classes = 18 # Number of classes
train_dataset = MultiLabelImageDataset(label_file='train1.csv', image_dir = 'data/', num_classes = 18, subset='train')
test_dataset = MultiLabelImageDataset(label_file='train1.csv', image_dir = 'data/', num_classes = 18, subset='test')



In [39]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [40]:

for layer in base_model.layers[:]:
    layer.trainable = False

In [41]:
dropout_rate = 0.1  # Set desired dropout rate
# Add custom classification layers
x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(1024, activation='relu')(x)
x = keras.layers.Dropout(dropout_rate)(x)
outputs = keras.layers.Dense(18, activation='sigmoid')(x)


In [42]:
# Create the final model
model = keras.models.Model(inputs=base_model.input, outputs=outputs)

In [43]:
# Compile the model with a custom learning rate
learning_rate = 0.00002
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [22]:
# Train the model
epochs = 3
model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=epochs,
    use_multiprocessing=False
)

Epoch 1/2
750/750 [==============================] - 1665s 2s/step - loss: 0.1186 - accuracy: 0.7888 - val_loss: 0.0962 - val_accuracy: 0.7947
Epoch 2/2
750/750 [==============================] - 1681s 2s/step - loss: 0.0977 - accuracy: 0.8025 - val_loss: 0.0924 - val_accuracy: 0.8142


In [32]:
model.save('model1.keras')

In [53]:
model = keras.models.load_model('model1.keras')
# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(test_dataset)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

188/188 [==============================] - 698s 4s/step - loss: 0.0927 - accuracy: 0.8177
Test Loss: 0.0927
Test Accuracy: 0.8177


In [54]:
# Generate predictions on the test dataset
predictions = model.predict(test_dataset)


188/188 [==============================] - 706s 4s/step


In [46]:
# Convert predictions to binary labels
threshold = 0.5
binary_predictions = np.where(predictions >= threshold, 1, 0)

# Get the true labels from the test dataset
true_labels = test_dataset.get_labels()

In [47]:
# Convert true labels to binary array format
mlb = MultiLabelBinarizer()
true_labels_binary = mlb.fit_transform(true_labels)

In [51]:
# Calculate additional evaluation metrics
f1 = f1_score(true_labels_binary, binary_predictions, average='micro')
#print(f'Precision: {precision:.4f}')
#print(f'Recall: {recall:.4f}')
print(f'F1-score: {f1:.4f}')

F1-score: 0.4506
